In [ ]:
# Unnecessary on google colab, run this to install on local
%conda install pytorch torchvision -c pytorch
%conda install -c fastai fastai

## Manual step-by-step

The code below has the parameters for the best model we were able to produce.

### Import and clone dataset

In [ ]:
from fastai.vision.all import *
from fastcore.all import *
from fastai.callback.tracker import EarlyStoppingCallback
from sklearn.metrics import f1_score
# !git clone https://github.com/davay/data5100.git # unnecessary on local

### Label and load images

In [ ]:
path = 'data' # local
# path = 'data5100/data' # google colab
dls = ImageDataLoaders.from_folder(path,
                            train = 'train',
                            valid = 'valid',
                            test = 'test',
                            item_tfms = Resize(450, pad_mode='zeros'),
                            # item_tfms = RandomResizedCrop(450, min_scale = 0.75), # imagenet models often use 224 x 224. Our images aren't 1:1 aspect ratio, by default center crop will be used. We can add pad_mode='zeros' for no cropping.
                            batch_tfms=[*aug_transforms(size=224, max_warp=0.), Normalize.from_stats(*imagenet_stats)],
                            bs=32) # default is 64, local runs out of memory when used with densenet201

### Retrain model on new data set

In [ ]:
learn = vision_learner(dls, densenet201, metrics=error_rate)
lr_valley = learn.lr_find().valley
epoch = 50
div = 25
learn.fit_one_cycle(epoch, lr_valley, div, cbs=[EarlyStoppingCallback(monitor='valid_loss', min_delta=0.01, patience=3)])

### Evaluate performance of retrained model

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix(figsize=(6,6))
interp.plot_top_losses(9, figsize=(15,10)) # TODO: fix overlapping text

In [ ]:
# Define a custom F1 score metric for Fastai
f1score_fastai = skm_to_fastai(f1_score, average='macro')

# Append the F1 score metric
learn.metrics.append(f1score_fastai)

# Validate and print F1 Score
val_f1score = learn.recorder.values[-1][2]  # Index 2 corresponds to the F1 score

print(f"F1 Score: {val_f1score}")

## Automated Runner

In [ ]:
from fastai.vision.all import *
from fastcore.all import *
from fastai.callback.tracker import EarlyStoppingCallback
from sklearn.metrics import f1_score
import gc 

path = 'data' # local
# path = 'data5100/data' # google colab

base_models = [
    resnet18,
    resnet50,
    resnet152,
    densenet121, 
    densenet201,
]

fit_options = [
    {"method": "fine_tune", "n_epoch": 50}, 
    {"method": "fit_one_cycle", "n_epoch": 50},
    {"method": "fit_one_cycle", "n_epoch": 50, "lr_max": "lr_find().valley", "early_stop": {"patience": 3}},
    {"method": "fit_one_cycle", "n_epoch": 50, "lr_max": "lr_find().valley", "div": 10, "early_stop": {"patience": 3}}, # div 10 is per resnet paper https://arxiv.org/abs/1512.03385
    {"method": "fit_one_cycle", "n_epoch": 50, "lr_max": "lr_find().valley", "early_stop": {"patience": 10}},
]

item_tfms_options = [
    "Resize(224)", # using default center crop
    "Resize(224, pad_mode='zeros')",
    "Resize(450, pad_mode='zeros')",
    "RandomResizedCrop(450, min_scale=0.75)" # from fastai paper https://arxiv.org/abs/2002.04688
    ]

batch_tfms_options = [
    "None",
    "aug_transforms()", 
    "[*aug_transforms(size=224, max_warp=0.), Normalize.from_stats(*imagenet_stats)]" # from fastai paper https://arxiv.org/abs/2002.04688
    ]

def evaluate(learner):
    f1score_fastai = skm_to_fastai(f1_score, average='macro')
    learner.metrics.append(f1score_fastai)
    f1 = learner.recorder.values[-1][2] # Index 2 corresponds to the F1 score
    return f1

# Iterate over all combinations
n = 1
for base_model in base_models:
    
    # Initialize new data frame per base model, we will separate the csv because there is a chance some runs intermittently fail after a few hundred runs
    results = pd.DataFrame(columns=["Base Model", "Fit Method", "n_epoch", "lr_max", "div", "patience", "item_tfms", "batch_tfms", "F1 Score"])

    for fit_option in fit_options:
        for batch_tfms in batch_tfms_options:
            for item_tfms in item_tfms_options:
                print("######################################")
                print(f"RUN: {n}")
                print(f"BASE MODEL: {base_model.__name__}") 
                print(f"FIT OPTION: {fit_option}")
                print(f"ITEM TFMS: {item_tfms}") 
                print(f"BATCH TFMS: {batch_tfms}")
                print("######################################")
                n += 1

                # Data Prep
                exec(f"item_tfms_obj = {item_tfms}") # Hacky way to grab the pre-evaluated option for cleaner output
                exec(f"batch_tfms_obj = {batch_tfms}")
                dls = ImageDataLoaders.from_folder(
                    path,
                    train='train',
                    valid='valid',
                    test='test',
                    item_tfms=item_tfms_obj,
                    batch_tfms=batch_tfms_obj,
                    bs=32) # default batch size is 64, local runs out of memory sometimes
                
                # Modelling
                learn = vision_learner(dls, base_model, metrics=error_rate)

                with learn.no_bar(), learn.no_logging():
                    lr_valley = learn.lr_find().valley if fit_option.get("lr_max", None) == "lr_find().valley" else None;
                    early_stop_params = fit_option.get("early_stop", None)
                    cbs = EarlyStoppingCallback(
                        monitor='valid_loss', 
                        min_delta=0.01, 
                        patience=early_stop_params.get("patience", 1) # default patience value is 1 - shouldn't get used tho
                    ) if early_stop_params else None
                    div = fit_option.get("div", 25)
                    if fit_option["method"] == "fit_one_cycle":
                        learn.fit_one_cycle(fit_option["n_epoch"], lr_valley, div=div if div else None, cbs=cbs if cbs else None)
                    if fit_option["method"] == "fine_tune":
                        learn.fine_tune(fit_option["n_epoch"])

                # Evaluation
                f1 = evaluate(learn)
                results = pd.concat([results, pd.DataFrame([{
                    "Base Model": base_model.__name__,
                    "Fit Method": fit_option["method"],
                    "n_epoch": fit_option["n_epoch"],
                    "lr_max": str(fit_option.get("lr_max", None)) + f" -> {lr_valley}",
                    "div": fit_option.get("div", 25),
                    "patience": fit_option["early_stop"]["patience"] if fit_option.get("early_stop", None) else 1, # TODO: TECHNICALLYYYY patience should be None in output if early stop dont exist, because it dont get used
                    "item_tfms": item_tfms,
                    "batch_tfms": batch_tfms,
                    "F1 Score": f1
                }])], ignore_index=True)
                
                # Reclaim GPU memory
                learn = None
                gc.collect()
                torch.cuda.empty_cache()

    results.to_csv(f"{base_model.__name__}.csv")

### Manually reclaim GPU memory

In [ ]:
import gc
learn = None
gc.collect()
torch.cuda.empty_cache()